In [10]:
import pandas as pd
import numpy as np

In [13]:
politics_data = pd.read_csv("politics_full.csv")
sports_data = pd.read_csv("sports_full.csv")
history_data = pd.read_csv("history_full.csv")
culture_data = pd.read_csv("culture_full.csv")
comp_science_data = pd.read_csv("comp_science_full.csv")

In [15]:
politics_data.head()

,Unnamed: 0,title,pageid,length,content
0,0,Newspaper endorsements in the United States pr...,51202493,523778,Various notable newspapers made endorsements o...
1,1,Timeline of 1960s counterculture,45684607,421412,The following is a chronological capsule histo...
2,2,Antisemitism,1078,211152,Antisemitism (also spelled anti-Semitism or an...
3,3,Libertarianism,3225498,172434,"Libertarianism (from Latin: libertas, meaning ..."
4,4,"List of scandals with \-gate\"" suffix""",243541,163074,This is a list of scandals or controversies wh...
